In [ ]:
import numpy as np
import pandas as pd
import math
import similaritymeasures
import sklearn
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import matplotlib
import sklearn
# from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv("standardized_trajs_best_perf_test_losses_indiv_traj.csv",header=[0, 1],index_col=[0, 1])
df

In [ ]:
optimizers = []
for i in df.columns:
    if i[0] not in optimizers:
        optimizers.append(i[0])
optimizers

In [ ]:
tasks = []
for i in df.index:
    if i[0] not in tasks:
        tasks.append(i[0])
tasks

In [ ]:
# x-axis range from 0 to 1
all_coordinates = []
for i in range(0,80,10):
    task_coordinates = []
    trajectory = list(df.iloc[i])
    for j in range(len(trajectory)):
        task_coordinates.append([3*j/len(trajectory),trajectory[j]])
    all_coordinates.append(task_coordinates)
all_coordinates

In [ ]:
# calculate distances

distances = []
for i in range(len(all_coordinates)):
    task_dist = []
    for j in range(len(all_coordinates)):
        task_dist.append(similaritymeasures.frechet_dist(all_coordinates[i],all_coordinates[j]))
    distances.append(task_dist)
distances

In [ ]:
# normalize distances?

test_losses = np.asarray(distances)
test_losses
normalized_test_losses = []

for i in range(len(test_losses)):
    mean = np.mean(test_losses[i])
    minus_mean = test_losses[i] - mean
    normalized_test_losses.append((minus_mean)/np.linalg.norm(minus_mean))
    
print(normalized_test_losses)

In [ ]:
results = pd.DataFrame(data=distances, index=tasks, columns=tasks)
results

In [ ]:
results.to_csv('frechet_dist1.csv')

In [ ]:
# only use y-coordinates for DTW

all_y_coordinates = []
for i in range(0,80,10):
    task_coordinates = []
    trajectory = list(df.iloc[i])
    for j in range(len(trajectory)):
        task_coordinates.append(trajectory[j])
    all_y_coordinates.append(task_coordinates)
all_y_coordinates

In [ ]:
# DTW plots

for i in range(len(all_y_coordinates)):
    for j in range(i, len(all_y_coordinates)):
        if i != j:
            for k in range(0, len(all_y_coordinates[i]), 51):
                s1 = all_y_coordinates[i][i:i+50]
                s2 = all_y_coordinates[j][i:i+50]
                path = dtw.warping_path(s1, s2)
                f = "plots/" + tasks[i] + "_" + tasks[j] + "_"+ optimizers[int(k/51)]
                dtwvis.plot_warping(s1, s2, path, filename = f)


In [ ]:
# calculate DTW distance

dtw_distances = []
for i in range(len(all_y_coordinates)):
    task_dist = []
    for j in range(len(all_y_coordinates)):
        task_dist.append(dtw.distance(all_y_coordinates[i],all_y_coordinates[j]))
    dtw_distances.append(task_dist)

In [ ]:
distances = pd.DataFrame(data=dtw_distances, index=tasks, columns=tasks)
distances

In [ ]:
distances.to_csv("dtw_distances.csv")